In [ ]:
from src import gads_client, config
from src.models.utils import choose_account_id

import sys
from google.ads.google_ads.errors import GoogleAdsException
import pandas as pd

# Put an account id to download stats from. Note: not MCC, no dash lines
CUSTOMER_ID = ""


def get_account_id(account_id, check_only=False):
    """
    Converts int to str, checks if str has dashes. Returns 10 chars str or raises error
    :check_only - if True, returns None instead of Error
    """

    if isinstance(account_id, int) and len(str(account_id)) == 10:
        return str(account_id)
    if isinstance(account_id, str) and len(account_id.replace("-", "")) == 10:
        return account_id.replace("-", "")
    if check_only:
        return None
    raise ValueError(f"Couldn't recognize account id from {account_id}")


def choose_account_id(account_id, test_account_id):
    """
    Picks actual account id or a test one. Check both of the accounts ids first.
    Raises value if both are incorrect
    """

    if not get_account_id(account_id, True) and not get_account_id(test_account_id, True):
        raise ValueError("Couldn't pick an account id ")
    return get_account_id(account_id, True) or get_account_id(test_account_id, True)


def micros_to_currency(micros):
    return micros / 1000000.0

def main(client, customer_id):
    ga_service = client.get_service("GoogleAdsService", version="v5")

    query = """
        SELECT
          campaign.id,
          campaign.name,
          ad_group.id,
          ad_group.name,
          ad_group_criterion.criterion_id,
          ad_group_criterion.keyword.text,
          ad_group_criterion.keyword.match_type,
          metrics.impressions,
          metrics.clicks,
          metrics.cost_micros
        FROM keyword_view
        WHERE
          segments.date DURING LAST_7_DAYS
          AND campaign.advertising_channel_type = 'SEARCH'
          AND ad_group.status = 'ENABLED'
          AND ad_group_criterion.status IN ('ENABLED', 'PAUSED')
        ORDER BY metrics.impressions DESC
        LIMIT 50"""

    # Issues a search request using streaming.
    response = ga_service.search_stream(customer_id, query)
    keyword_match_type_enum = client.get_type(
        "KeywordMatchTypeEnum", version="v5"
    ).KeywordMatchType
    
    all_data=[]
    try:
        for batch in response:
            for row in batch.results:
                single_row = dict()
                single_row['campaign_name'] = row.campaign.name
                single_row['ad_group_name'] = row.ad_group.name
                single_row['keyword'] = row.ad_group_criterion.keyword.text
                single_row['keyword_matchtype'] = row.ad_group_criterion.keyword.match_type
                single_row['impressions'] = row.metrics.impressions
                single_row['clicks'] = row.metrics.clicks
                single_row['cost'] = row.metrics.cost_micros
                print(single_row)
                
                all_data.append(single_row)
    except GoogleAdsException as ex:
        print(
            f'Request with ID "{ex.request_id}" failed with status '
            f'"{ex.error.code().name}" and includes the following errors:'
        )
        for error in ex.failure.errors:
            print(f'\tError with message "{error.message}".')
            if error.location:
                for field_path_element in error.location.field_path_elements:
                    print(f"\t\tOn field: {field_path_element.field_name}")
        sys.exit(1)
    df = pd.DataFrame(all_data)
    df.to_csv('google_ads_keywords'.csv)


if __name__ == "__main__":
    id_to_load = choose_account_id(CUSTOMER_ID, config.get("test_account_id", None))
    main(gads_client, id_to_load)